In [ ]:
# google colab

!pip install folium pandas

# Map

This section focuses on the getting of data and creation of the map

https://apex.oracle.com/pls/apex/hackathonjune2024/NilePProject/GpsData?gpsId=1&longi=43.434&lati=434.3443&TimeStamp=2024-06-12 09:43:32&targetOBJ=Boat&gpsOrigin=real

In [ ]:
import requests

url = 'https://apex.oracle.com/pls/apex/hackathonjune2024/NilePProject/GpsData?gpsId=1'

r = requests.get(url)


print(r.status_code)
print("ok")

def get_request():
    # implement request and store response as json
    pass

In [ ]:
import json
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [ ]:
fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map


def create_geolocation():
    coords_all = df[["lat", "long"]].reset_index(drop=True)
    coords_start = [df.loc[0, "lat"], df.loc[0, "long"]]
    time_start = df.loc[0, "timestamp"]
    coord_last = [df.loc[(len(df) - 1), "lat"], df.loc[(len(df) - 1), "long"]]
    time_last = df.loc[(len(df) - 1), "timestamp"]
    vehicle_last = df.loc[(len(df) - 1), "tracked_object"]
        
    basemap = folium.Map(location = coord_last, zoom_start = 13)
    # folium.PolyLine(coord_list).add_to(basemap)
    folium.PolyLine(coords_all).add_to(basemap)
    fg.add_to(basemap)
    folium.LayerControl(position='bottomright').add_to(basemap)

    folium.Marker(coords_start, popup = f'Started at time: {time_start}', color = "red").add_to(basemap)
    folium.Marker(coord_last, popup = f'Last Seen at time: {time_last}\nVehicle: {vehicle_last}', color = "blue").add_to(basemap)

    basemap.save("map.html")

def convert_json_to_dataframe(json_object):
    return pd.read_json(json_object)


df = convert_json_to_dataframe("20240612_162723_tracking_data.json")

create_geolocation()

In [ ]:
import webbrowser

webbrowser.open("map.html")

# Testing

In [1]:
# testing ground to implement different color lines for each transportation used

In [ ]:
fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map
coords = []
startpoint_marker = []
endpoint_marker = []


def create_geolocation(coord_list: list):
  print(coord_list[(len(coord_list) - 1)][0])
  idx_nr = (len(coord_list) - 1)
  try:
    basemap = folium.Map(location = [coord_list[idx_nr][0], coord_list[idx_nr][1]], zoom_start = 13)
  except:
    print(f"basemap error")


  for x in range(len(coord_list)):
    try:
      folium.PolyLine([coord_list[0], coord_list[1]], color = coord_list[2]).add_to(basemap)
    except:
      print(f"Possible mismatch while creating route, coord:[ {coord_list[x][0]}, {coord_list[x][1]} ],and color: {coord_list[2]}")

  fg.add_to(basemap)
  folium.LayerControl(position='bottomright').add_to(basemap)


  folium.Marker([startpoint_marker[0][0], startpoint_marker[0][1]], popup = 'Starting Point', color = "red").add_to(basemap)
  folium.Marker([endpoint_marker[0][0], endpoint_marker[0][1]], popup = 'Last Point Seen', color = "blue").add_to(basemap)
  basemap.save("line.html")
  clear_lists()


def convert_json_to_dataframe(json_object):
  return pd.read_json(json_object)


def get_geolocation_from_df(dataframe: object):
  for index, row in df.iterrows():
    if row["tracked_object"] == "boat":
      coords.append([row["lat"], row["long"], "blue"])
    if row["tracked_object"] == "plane":
      coords.append([row["lat"], row["long"], "yellow"])
    if row["tracked_object"] == "truck":
      coords.append([row["lat"], row["long"], "green"])

  startpoint_marker.append(coords[0])
  endpoint_marker.append(coords[(len(coords) - 1)])


def clear_lists():
  coords.clear()
  startpoint_marker.clear()
  endpoint_marker.clear()

df = convert_json_to_dataframe("mocked_tracking_data_4.json")

In [ ]:
get_geolocation_from_df(df)
create_geolocation(coords)